In [ ]:
import pandas as pd
import os 
desktop_path = os.path.join(os.path.expanduser("~"), "llm-justification-evaluation", "Data_cleaning_cosine_calculation_semantic_and_analysis")
os.chdir(desktop_path)

In [5]:
math_data = pd.read_csv("NLP_analysis/math_analysis.csv")
math_data

,uuid,deepseek-r1:1.5b,deepseek-r1:14b,qwen2.5:1.5b,qwen2.5:14b,deepseek-r1:1.5b_time,deepseek-r1:14b_time,qwen2.5:1.5b_time,qwen2.5:14b_time,problem,...,qwen2.5:1.5b_reasoning,qwen2.5:1.5b_solution,qwen2.5:1.5b_structure_score,qwen2.5:14b_reasoning,qwen2.5:14b_solution,qwen2.5:14b_structure_score,deepseek-r1:1.5b_correct,deepseek-r1:14b_correct,qwen2.5:1.5b_correct,qwen2.5:14b_correct
0,4e2acd29-7c56-51af-8c62-6741929c5e64,## Reasoning\n\nThe problem requires determini...,## Reasoning\nLet $a$ and $b$ be lines such th...,"To solve this problem, we need to analyze the ...",## Reasoning\n\nTo determine the relationship ...,16.7,41.2,5.0,96.3,"Given lines $a$ and $b$, and planes $\alpha$ a...",...,"To solve this problem, we need to analyze the ...",C,1,To determine the relationship between the cond...,B,0,0,1,0,1
1,fb45369a-73d7-53c9-b6b3-1614075d5d35,## Reasoning\n\nTo find the GCD of the two pro...,The greatest common divisor (GCD) of the produ...,## Reasoning\nTo find the greatest common divi...,## Reasoning\n\nTo find the greatest common di...,7.9,171.6,3.4,11.5,The greatest common divisor of the products $3...,...,To find the greatest common divisor of \(360 \...,B,0,To find the greatest common divisor (GCD) of t...,C,0,0,0,0,1
2,7c97406b-7297-5ada-b57f-4294f81617bb,## Reasoning\nTo find the number of points \(P...,To determine the number of points \( P \) on t...,## Reasoning\nTo find the number of points \( ...,## Reasoning\n\nTo determine how many points \...,16.9,158.7,10.4,111.5,4 The line $\frac{x}{4}+\frac{y}{3}=1$ interse...,...,To find the number of points \( P \) on the el...,B,0,To determine how many points \( P \) exist on ...,D,0,1,1,1,0
3,b5f661d8-4106-5b36-8d05-b5d3b02dfc98,"## Reasoning\nLet \(A = \{1, 3\}\) and \(B = \...","## Reasoning\nThe union of sets $A$ and $B$, d...",## Reasoning\nTo determine the union of sets $...,## Reasoning\nThe union of two sets $A$ and $B...,108.0,18.5,1.5,5.9,"Let set $A=\{1,3\}$, and set $B=\{1,2,4,5\}$. ...",...,"To determine the union of sets $A$ and $B$, we...",C,0,"The union of two sets $A$ and $B$, denoted as ...",C,0,1,1,1,1
4,5695a817-033f-5e9c-997a-dd14d968b772,"## Reasoning\nTo solve the problem, we first f...","## Reasoning\nThe universal set is \( U = \{1,...","## Reasoning\n\nTo solve this problem, we need...",## Reasoning\nTo solve the problem $N \cap (\c...,152.0,16.1,2.7,6.8,"Let the universal set $U=\{1,2,3,4,5\}$, set $...",...,"To solve this problem, we need to find the int...",B,1,To solve the problem $N \cap (\complement_U M)...,C,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,46a318c6-abdd-546c-a84a-0483759417f7,## Solution\nThe hyperbola equation is (x²/m²)...,## Reasoning\nThe given hyperbola equation is ...,## Reasoning\nThe standard form of a hyperbola...,## Reasoning\nThe equation of the hyperbola is...,27.8,33.2,4.4,126.4,Given that the focal length of the hyperbola $...,...,The standard form of a hyperbola is $\frac{x^2...,A,0,The equation of the hyperbola is given by:\n\[...,C,0,0,1,0,0
496,ddbc339f-0566-521e-95b6-4f4e135dd7f5,"## Reasoning\nTo solve the problem, we need to...",## Reasoning\nTo determine \( a \) such that \...,"## Reasoning\n\nTo solve this problem, we need...",## Reasoning\nTo determine the value of \( a \...,7.5,289.7,3.9,13.3,"Let $a\in Z$, and $0\leq a\leq 13$. If $51^{20...",...,"To solve this problem, we need to find the rem...",A,0,To determine the value of \( a \) such that \(...,B,0,1,1,0,1
497,b50b4540-a93c-598a-a6e0-e15edab5f711,## Reasoning\n\nTo ensure that the function \(...,To determine the values of \( k \) such that t...,To determine the range of \( k \) for which th...,## Reasoning\nTo determine the range of \( k \...,13.9,215.9,5.9,98.7,Given the function $y=\log_{2}(x^{2}-2kx+k)$ h...,...,To determine the range of \( k \) for which th...,D,1,To determine the range of \( k \) such that th...,C,0,1,0,0,1
498,9ec52114-adca-53ef-a2ab-47a7ec5a8670,"The sides of

In [6]:
math_data.isna().sum()
math_data.dropna(inplace=True)

In [7]:
corpus = math_data["solution"].tolist() + \
         math_data["deepseek-r1:1.5b_reasoning"].tolist() + \
         math_data["deepseek-r1:14b_reasoning"].tolist() + \
         math_data["qwen2.5:1.5b_reasoning"].tolist() + \
         math_data["qwen2.5:14b_reasoning"].tolist()

In [8]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word',
                             min_df=0.01,
                             max_df=0.75,
                             ngram_range=(1, 2),
                             stop_words='english',
                             sublinear_tf=True)

X = vectorizer.fit_transform(corpus)


In [9]:
n = len(math_data)

Solution = X[0:n]
deepseek_r1_1_5b = X[n:2*n]
deepseek_r1_14b = X[2*n:3*n]
qwen2_5_1_5b = X[3*n:4*n]
qwen2_5_14b = X[4*n:5*n]

In [10]:
# Cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

deepseek_r1_1_5b_similarity = cosine_similarity(Solution, deepseek_r1_1_5b).diagonal()
deepseek_r1_14b_similarity = cosine_similarity(Solution, deepseek_r1_14b).diagonal()
qwen2_5_1_5b_similarity = cosine_similarity(Solution, qwen2_5_1_5b).diagonal()
qwen2_5_14b_similarity = cosine_similarity(Solution, qwen2_5_14b).diagonal()

In [11]:
def mean_similarity_by_correctness(similarity, model_solution_col):
    correct_mask = math_data[model_solution_col] == math_data['answer']
    mean_correct = similarity[correct_mask.values].mean()
    mean_wrong = similarity[~correct_mask.values].mean()
    return mean_correct, mean_wrong

In [12]:
mean_corr_dk1, mean_wrong_dk1 = mean_similarity_by_correctness(deepseek_r1_1_5b_similarity, 'deepseek-r1:1.5b_solution')
mean_corr_dk14, mean_wrong_dk14 = mean_similarity_by_correctness(deepseek_r1_14b_similarity, 'deepseek-r1:14b_solution')
mean_corr_qw1, mean_wrong_qw1 = mean_similarity_by_correctness(qwen2_5_1_5b_similarity, 'qwen2.5:1.5b_solution')
mean_corr_qw14, mean_wrong_qw14 = mean_similarity_by_correctness(qwen2_5_14b_similarity, 'qwen2.5:14b_solution')

print("DeepSeek R1 1.5B - Mean similarity (correct):", mean_corr_dk1)
print("DeepSeek R1 1.5B - Mean similarity (wrong):", mean_wrong_dk1)
print("DeepSeek R1 14B - Mean similarity (correct):", mean_corr_dk14)
print("DeepSeek R1 14B - Mean similarity (wrong):", mean_wrong_dk14)
print("Qwen2.5 1.5B - Mean similarity (correct):", mean_corr_qw1)
print("Qwen2.5 1.5B - Mean similarity (wrong):", mean_wrong_qw1)
print("Qwen2.5 14B - Mean similarity (correct):", mean_corr_qw14)
print("Qwen2.5 14B - Mean similarity (wrong):", mean_wrong_qw14)

DeepSeek R1 1.5B - Mean similarity (correct): 0.37526300977404453
DeepSeek R1 1.5B - Mean similarity (wrong): 0.2959716711842935
DeepSeek R1 14B - Mean similarity (correct): 0.4138669042304432
DeepSeek R1 14B - Mean similarity (wrong): 0.25763420054918945
Qwen2.5 1.5B - Mean similarity (correct): 0.44052758646254436
Qwen2.5 1.5B - Mean similarity (wrong): 0.3978871219955683
Qwen2.5 14B - Mean similarity (correct): 0.44160221677382105
Qwen2.5 14B - Mean similarity (wrong): 0.36665108185266276


In [13]:
# Mean cosine similarities
print("Mean Cosine Similarity (DeepSeek 1.5B):", deepseek_r1_1_5b_similarity.mean())
print("Mean Cosine Similarity (DeepSeek 14B):", deepseek_r1_14b_similarity.mean())
print("Mean Cosine Similarity (Qwen 1.5B):", qwen2_5_1_5b_similarity.mean())
print("Mean Cosine Similarity (Qwen 14B):", qwen2_5_14b_similarity.mean())

Mean Cosine Similarity (DeepSeek 1.5B): 0.34262333193460437
Mean Cosine Similarity (DeepSeek 14B): 0.3862582143907206
Mean Cosine Similarity (Qwen 1.5B): 0.4161489425365186
Mean Cosine Similarity (Qwen 14B): 0.4252407424147324


In [14]:
import numpy as np


stats = {
    "Model": [
        "DeepSeek R1 1.5B",
        "DeepSeek R1 14B",
        "Qwen2.5 1.5B",
        "Qwen2.5 14B"
    ],
    "Mean": [
        np.mean(deepseek_r1_1_5b_similarity),
        np.mean(deepseek_r1_14b_similarity),
        np.mean(qwen2_5_1_5b_similarity),
        np.mean(qwen2_5_14b_similarity)
    ],
    "Median": [
        np.median(deepseek_r1_1_5b_similarity),
        np.median(deepseek_r1_14b_similarity),
        np.median(qwen2_5_1_5b_similarity),
        np.median(qwen2_5_14b_similarity)
    ],
    "Std": [
        np.std(deepseek_r1_1_5b_similarity),
        np.std(deepseek_r1_14b_similarity),
        np.std(qwen2_5_1_5b_similarity),
        np.std(qwen2_5_14b_similarity)
    ]
}

stats_df = pd.DataFrame(stats)
print(stats_df)

              Model      Mean    Median       Std
0  DeepSeek R1 1.5B  0.342623  0.344229  0.213919
1   DeepSeek R1 14B  0.386258  0.404378  0.214303
2      Qwen2.5 1.5B  0.416149  0.425770  0.191579
3       Qwen2.5 14B  0.425241  0.442808  0.193833


N-GRAMS OVERLAP

In [15]:
### N-GRAM OVERLAP ###
def ngrams(text, n):
    tokens = text.lower().split()
    return set([' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)])

def ngram_overlap(text1, text2, n):
    ngrams1 = ngrams(text1, n)
    ngrams2 = ngrams(text2, n)
    
    intersection = len(ngrams1 & ngrams2)
    union = len(ngrams1 | ngrams2)
    
    if union == 0:
        return 0.0
    else:
        return intersection / union

n = 3

In [16]:
math_data['deepseek_r1_1_5b_ngram_overlap'] = math_data.apply(lambda row: ngram_overlap(row['solution'], row['deepseek-r1:1.5b_reasoning'], n), axis=1)
math_data['deepseek_r1_14b_ngram_overlap']  = math_data.apply(lambda row: ngram_overlap(row['solution'], row['deepseek-r1:14b_reasoning'], n), axis=1)
math_data['qwen2_5_1_5b_ngram_overlap']     = math_data.apply(lambda row: ngram_overlap(row['solution'], row['qwen2.5:1.5b_reasoning'], n), axis=1)
math_data['qwen2_5_14b_ngram_overlap']      = math_data.apply(lambda row: ngram_overlap(row['solution'], row['qwen2.5:14b_reasoning'], n), axis=1)

In [17]:
# DeepSeek R1 1.5B
mean_corr_dk1, mean_wrong_dk1 = mean_similarity_by_correctness(math_data['deepseek_r1_1_5b_ngram_overlap'], 'deepseek-r1:1.5b_solution')
print("DeepSeek R1 1.5B - Mean overlap (correct):", mean_corr_dk1)
print("DeepSeek R1 1.5B - Mean overlap (wrong):", mean_wrong_dk1)

# DeepSeek R1 14B
mean_corr_dk14, mean_wrong_dk14 = mean_similarity_by_correctness(math_data['deepseek_r1_14b_ngram_overlap'], 'deepseek-r1:14b_solution')
print("DeepSeek R1 14B - Mean overlap (correct):", mean_corr_dk14)
print("DeepSeek R1 14B - Mean overlap (wrong):", mean_wrong_dk14)

# Qwen2.5 1.5B
mean_corr_qw1, mean_wrong_qw1 = mean_similarity_by_correctness(math_data['qwen2_5_1_5b_ngram_overlap'], 'qwen2.5:1.5b_solution')
print("Qwen2.5 1.5B - Mean overlap (correct):", mean_corr_qw1)
print("Qwen2.5 1.5B - Mean overlap (wrong):", mean_wrong_qw1)

# Qwen2.5 14B
mean_corr_qw14, mean_wrong_qw14 = mean_similarity_by_correctness(math_data['qwen2_5_14b_ngram_overlap'], 'qwen2.5:14b_solution')
print("Qwen2.5 14B - Mean overlap (correct):", mean_corr_qw14)
print("Qwen2.5 14B - Mean overlap (wrong):", mean_wrong_qw14)

DeepSeek R1 1.5B - Mean overlap (correct): 0.022604373152206002
DeepSeek R1 1.5B - Mean overlap (wrong): 0.012452923394151324
DeepSeek R1 14B - Mean overlap (correct): 0.023913523511951865
DeepSeek R1 14B - Mean overlap (wrong): 0.013440504644776015
Qwen2.5 1.5B - Mean overlap (correct): 0.029871061308188944
Qwen2.5 1.5B - Mean overlap (wrong): 0.020804095154224864
Qwen2.5 14B - Mean overlap (correct): 0.027929082382956185
Qwen2.5 14B - Mean overlap (wrong): 0.014266007020642932


In [18]:
# Mean N-gram overlaps
print("Mean N-gram overlap (DeepSeek 1.5B):", math_data['deepseek_r1_1_5b_ngram_overlap'].mean())
print("Mean N-gram overlap (DeepSeek 14B):", math_data['deepseek_r1_14b_ngram_overlap'].mean())
print("Mean N-gram overlap (Qwen 1.5B):", math_data['qwen2_5_1_5b_ngram_overlap'].mean())
print("Mean N-gram overlap (Qwen 14B):", math_data['qwen2_5_14b_ngram_overlap'].mean())

Mean N-gram overlap (DeepSeek 1.5B): 0.0184256058921336
Mean N-gram overlap (DeepSeek 14B): 0.022062782132097506
Mean N-gram overlap (Qwen 1.5B): 0.02468724489999742
Mean N-gram overlap (Qwen 14B): 0.024946498364155995


METRICHE SCORE

In [19]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import single_meteor_score
from rouge_score import rouge_scorer
import sacrebleu

smoothie = SmoothingFunction().method4

In [20]:
def compute_bleu(reference_text, candidate_text):
    reference = [reference_text.lower().split()]
    candidate = candidate_text.lower().split()
    return sentence_bleu(reference, candidate, smoothing_function=smoothie)

In [21]:
### METEOR SCORE ###

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

def compute_meteor(reference_text, candidate_text):
    return single_meteor_score(reference_text.lower().split(), candidate_text.lower().split())

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [22]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge(reference_text, candidate_text):
    score = scorer.score(reference_text, candidate_text)
    return score['rougeL'].fmeasure

In [23]:
# BLEU example DeepSeek 1.5B
math_data['deepseek_r1_1_5b_bleu'] = math_data.apply(lambda row: compute_bleu(row['solution'], row['deepseek-r1:1.5b_reasoning']), axis=1)
math_data['deepseek_r1_14b_bleu'] = math_data.apply(lambda row: compute_bleu(row['solution'], row['deepseek-r1:14b_reasoning']), axis=1)
math_data['qwen2_5_1_5b_bleu'] = math_data.apply(lambda row: compute_bleu(row['solution'], row['qwen2.5:1.5b_reasoning']), axis=1)
math_data['qwen2_5_14b_bleu'] = math_data.apply(lambda row: compute_bleu(row['solution'], row['qwen2.5:14b_reasoning']), axis=1)

In [24]:
# METEOR
math_data['deepseek_r1_1_5b_meteor'] = math_data.apply(lambda row: compute_meteor(row['solution'], row['deepseek-r1:1.5b_reasoning']), axis=1)
math_data['deepseek_r1_14b_meteor'] = math_data.apply(lambda row: compute_meteor(row['solution'], row['deepseek-r1:14b_reasoning']), axis=1)
math_data['qwen2_5_1_5b_meteor'] = math_data.apply(lambda row: compute_meteor(row['solution'], row['qwen2.5:1.5b_reasoning']), axis=1)
math_data['qwen2_5_14b_meteor'] = math_data.apply(lambda row: compute_meteor(row['solution'], row['qwen2.5:14b_reasoning']), axis=1)

In [25]:
# ROUGE-L
math_data['deepseek_r1_1_5b_rouge'] = math_data.apply(lambda row: compute_rouge(row['solution'], row['deepseek-r1:1.5b_reasoning']), axis=1)
math_data['deepseek_r1_14b_rouge'] = math_data.apply(lambda row: compute_rouge(row['solution'], row['deepseek-r1:14b_reasoning']), axis=1)
math_data['qwen2_5_1_5b_rouge'] = math_data.apply(lambda row: compute_rouge(row['solution'], row['qwen2.5:1.5b_reasoning']), axis=1)
math_data['qwen2_5_14b_rouge'] = math_data.apply(lambda row: compute_rouge(row['solution'], row['qwen2.5:14b_reasoning']), axis=1)

In [26]:
# BLEU
mean_bleu_corr_dk1, mean_bleu_wrong_dk1 = mean_similarity_by_correctness(math_data['deepseek_r1_1_5b_bleu'], 'deepseek-r1:1.5b_solution')
mean_bleu_corr_dk14, mean_bleu_wrong_dk14 = mean_similarity_by_correctness(math_data['deepseek_r1_14b_bleu'], 'deepseek-r1:14b_solution')
mean_bleu_corr_qw1, mean_bleu_wrong_qw1 = mean_similarity_by_correctness(math_data['qwen2_5_1_5b_bleu'], 'qwen2.5:1.5b_solution')
mean_bleu_corr_qw14, mean_bleu_wrong_qw14 = mean_similarity_by_correctness(math_data['qwen2_5_14b_bleu'], 'qwen2.5:14b_solution')

print("BLEU - DeepSeek R1 1.5B (correct):", mean_bleu_corr_dk1)
print("BLEU - DeepSeek R1 1.5B (wrong):", mean_bleu_wrong_dk1)
print("BLEU - DeepSeek R1 14B (correct):", mean_bleu_corr_dk14)
print("BLEU - DeepSeek R1 14B (wrong):", mean_bleu_wrong_dk14)
print("BLEU - Qwen2.5 1.5B (correct):", mean_bleu_corr_qw1)
print("BLEU - Qwen2.5 1.5B (wrong):", mean_bleu_wrong_qw1)
print("BLEU - Qwen2.5 14B (correct):", mean_bleu_corr_qw14)
print("BLEU - Qwen2.5 14B (wrong):", mean_bleu_wrong_qw14)

# ROUGE-L
mean_rouge_corr_dk1, mean_rouge_wrong_dk1 = mean_similarity_by_correctness(math_data['deepseek_r1_1_5b_rouge'], 'deepseek-r1:1.5b_solution')
mean_rouge_corr_dk14, mean_rouge_wrong_dk14 = mean_similarity_by_correctness(math_data['deepseek_r1_14b_rouge'], 'deepseek-r1:14b_solution')
mean_rouge_corr_qw1, mean_rouge_wrong_qw1 = mean_similarity_by_correctness(math_data['qwen2_5_1_5b_rouge'], 'qwen2.5:1.5b_solution')
mean_rouge_corr_qw14, mean_rouge_wrong_qw14 = mean_similarity_by_correctness(math_data['qwen2_5_14b_rouge'], 'qwen2.5:14b_solution')

print("ROUGE-L - DeepSeek R1 1.5B (correct):", mean_rouge_corr_dk1)
print("ROUGE-L - DeepSeek R1 1.5B (wrong):", mean_rouge_wrong_dk1)
print("ROUGE-L - DeepSeek R1 14B (correct):", mean_rouge_corr_dk14)
print("ROUGE-L - DeepSeek R1 14B (wrong):", mean_rouge_wrong_dk14)
print("ROUGE-L - Qwen2.5 1.5B (correct):", mean_rouge_corr_qw1)
print("ROUGE-L - Qwen2.5 1.5B (wrong):", mean_rouge_wrong_qw1)
print("ROUGE-L - Qwen2.5 14B (correct):", mean_rouge_corr_qw14)
print("ROUGE-L - Qwen2.5 14B (wrong):", mean_rouge_wrong_qw14)

# METEOR
mean_meteor_corr_dk1, mean_meteor_wrong_dk1 = mean_similarity_by_correctness(math_data['deepseek_r1_1_5b_meteor'], 'deepseek-r1:1.5b_solution')
mean_meteor_corr_dk14, mean_meteor_wrong_dk14 = mean_similarity_by_correctness(math_data['deepseek_r1_14b_meteor'], 'deepseek-r1:14b_solution')
mean_meteor_corr_qw1, mean_meteor_wrong_qw1 = mean_similarity_by_correctness(math_data['qwen2_5_1_5b_meteor'], 'qwen2.5:1.5b_solution')
mean_meteor_corr_qw14, mean_meteor_wrong_qw14 = mean_similarity_by_correctness(math_data['qwen2_5_14b_meteor'], 'qwen2.5:14b_solution')

print("METEOR - DeepSeek R1 1.5B (correct):", mean_meteor_corr_dk1)
print("METEOR - DeepSeek R1 1.5B (wrong):", mean_meteor_wrong_dk1)
print("METEOR - DeepSeek R1 14B (correct):", mean_meteor_corr_dk14)
print("METEOR - DeepSeek R1 14B (wrong):", mean_meteor_wrong_dk14)
print("METEOR - Qwen2.5 1.5B (correct):", mean_meteor_corr_qw1)
print("METEOR - Qwen2.5 1.5B (wrong):", mean_meteor_wrong_qw1)
print("METEOR - Qwen2.5 14B (correct):", mean_meteor_corr_qw14)
print("METEOR - Qwen2.5 14B (wrong):", mean_meteor_wrong_qw14)

BLEU - DeepSeek R1 1.5B (correct): 0.04785008581853584
BLEU - DeepSeek R1 1.5B (wrong): 0.028247988393723448
BLEU - DeepSeek R1 14B (correct): 0.04839912516462233
BLEU - DeepSeek R1 14B (wrong): 0.029382865948273176
BLEU - Qwen2.5 1.5B (correct): 0.06150864468785496
BLEU - Qwen2.5 1.5B (wrong): 0.04576780760984827
BLEU - Qwen2.5 14B (correct): 0.06251646485038574
BLEU - Qwen2.5 14B (wrong): 0.03138418794928011
ROUGE-L - DeepSeek R1 1.5B (correct): 0.2845554937643426
ROUGE-L - DeepSeek R1 1.5B (wrong): 0.21388989108920672
ROUGE-L - DeepSeek R1 14B (correct): 0.2939320052175771
ROUGE-L - DeepSeek R1 14B (wrong): 0.1908158080470654
ROUGE-L - Qwen2.5 1.5B (correct): 0.2989456973259403
ROUGE-L - Qwen2.5 1.5B (wrong): 0.2559822868681924
ROUGE-L - Qwen2.5 14B (correct): 0.28492240863343943
ROUGE-L - Qwen2.5 14B (wrong): 0.2167373284328777
METEOR - DeepSeek R1 1.5B (correct): 0.1766260379090399
METEOR - DeepSeek R1 1.5B (wrong): 0.1469280765969044
METEOR - DeepSeek R1 14B (correct): 0.18578029

In [27]:
print("Mean BLEU (DeepSeek 1.5B):", math_data['deepseek_r1_1_5b_bleu'].mean())
print("Mean BLEU (DeepSeek 14B):", math_data['deepseek_r1_14b_bleu'].mean())
print("Mean BLEU (Qwen 1.5B):", math_data['qwen2_5_1_5b_bleu'].mean())
print("Mean BLEU (Qwen 14B):", math_data['qwen2_5_14b_bleu'].mean())

print("Mean METEOR (DeepSeek 1.5B):", math_data['deepseek_r1_1_5b_meteor'].mean())
print("Mean METEOR (DeepSeek 14B):", math_data['deepseek_r1_14b_meteor'].mean())
print("Mean METEOR (Qwen 1.5B):", math_data['qwen2_5_1_5b_meteor'].mean())
print("Mean METEOR (Qwen 14B):", math_data['qwen2_5_14b_meteor'].mean())

print("Mean ROUGE-L (DeepSeek 1.5B):", math_data['deepseek_r1_1_5b_rouge'].mean())
print("Mean ROUGE-L (DeepSeek 14B):", math_data['deepseek_r1_14b_rouge'].mean())
print("Mean ROUGE-L (Qwen 1.5B):", math_data['qwen2_5_1_5b_rouge'].mean())
print("Mean ROUGE-L (Qwen 14B):", math_data['qwen2_5_14b_rouge'].mean())

Mean BLEU (DeepSeek 1.5B): 0.039781031161336564
Mean BLEU (DeepSeek 14B): 0.04503866355674358
Mean BLEU (Qwen 1.5B): 0.052509205609992504
Mean BLEU (Qwen 14B): 0.0557204376682317
Mean METEOR (DeepSeek 1.5B): 0.16440109749364942
Mean METEOR (DeepSeek 14B): 0.1776734596518598
Mean METEOR (Qwen 1.5B): 0.22904577471026596
Mean METEOR (Qwen 14B): 0.2372396004794695
Mean ROUGE-L (DeepSeek 1.5B): 0.2554665346589852
Mean ROUGE-L (DeepSeek 14B): 0.2757098082123931
Mean ROUGE-L (Qwen 1.5B): 0.274382416918704
Mean ROUGE-L (Qwen 14B): 0.27003793166658085


In [28]:
metrics = {
    "Cosine Similarity": [
        deepseek_r1_1_5b_similarity.mean(),
        deepseek_r1_14b_similarity.mean(),
        qwen2_5_1_5b_similarity.mean(),
        qwen2_5_14b_similarity.mean()
    ],
    "N-gram Overlap": [
        math_data['deepseek_r1_1_5b_ngram_overlap'].mean(),
        math_data['deepseek_r1_14b_ngram_overlap'].mean(),
        math_data['qwen2_5_1_5b_ngram_overlap'].mean(),
        math_data['qwen2_5_14b_ngram_overlap'].mean()
    ],
    "BLEU": [
        math_data['deepseek_r1_1_5b_bleu'].mean(),
        math_data['deepseek_r1_14b_bleu'].mean(),
        math_data['qwen2_5_1_5b_bleu'].mean(),
        math_data['qwen2_5_14b_bleu'].mean()
    ],
    "ROUGE-L": [
        math_data['deepseek_r1_1_5b_rouge'].mean(),
        math_data['deepseek_r1_14b_rouge'].mean(),
        math_data['qwen2_5_1_5b_rouge'].mean(),
        math_data['qwen2_5_14b_rouge'].mean()
    ],
    "METEOR": [
        math_data['deepseek_r1_1_5b_meteor'].mean(),
        math_data['deepseek_r1_14b_meteor'].mean(),
        math_data['qwen2_5_1_5b_meteor'].mean(),
        math_data['qwen2_5_14b_meteor'].mean()
    ]
}

models = ["DeepSeek R1 1.5B", "DeepSeek R1 14B", "Qwen2.5 1.5B", "Qwen2.5 14B"]
metrics_df = pd.DataFrame(metrics, index=models)
print(metrics_df)

                  Cosine Similarity  N-gram Overlap      BLEU   ROUGE-L  \
DeepSeek R1 1.5B           0.342623        0.018426  0.039781  0.255467   
DeepSeek R1 14B            0.386258        0.022063  0.045039  0.275710   
Qwen2.5 1.5B               0.416149        0.024687  0.052509  0.274382   
Qwen2.5 14B                0.425241        0.024946  0.055720  0.270038   

                    METEOR  
DeepSeek R1 1.5B  0.164401  
DeepSeek R1 14B   0.177673  
Qwen2.5 1.5B      0.229046  
Qwen2.5 14B       0.237240  
